# Toronto Dwellings Analysis Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions to create a Panel dashboard.

In [109]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv

In [2]:
# Initialize the Panel Extensions (for Plotly)
import panel as pn
pn.extension("plotly")

In [3]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [4]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/toronto_neighbourhoods_census_data.csv")
to_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/toronto_neighbourhoods_coordinates.csv")
df_neighbourhood_locations = pd.read_csv(file_path,index_col='neighbourhood')

file_path = Path("sum_dwelling.csv")
sum_dwelling = pd.read_csv(file_path, index_col="year")

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

### Global available data

In [5]:
# Getting the data from the top 10 expensive neighbourhoods
# YOUR CODE HERE!

richest_neighbourhoods = to_data.groupby('neighbourhood').mean()
richest_neighbourhoods_sorted = richest_neighbourhoods.sort_values('average_house_value', ascending=False)
richest_neighbourhoods_sorted_top_10 = richest_neighbourhoods_sorted[0:10]
richest_neighbourhoods_sorted_top_10_housevalue = richest_neighbourhoods_sorted_top_10['average_house_value']

In [6]:
# Join the average values with the neighbourhood locations
# YOUR CODE HERE!

df_mean_neighbourhood_locations = pd.concat([richest_neighbourhoods, df_neighbourhood_locations], axis=1)

In [7]:
# Calculate the mean number of dwelling types units per year
# YOUR CODE HERE!

to_data_reset_plot = to_data.groupby(['year', 'neighbourhood']).mean()

In [8]:
# Calculate the average monthly shelter costs for owned and rented dwellings
# YOUR CODE HERE!

costs_mean = to_data_reset_plot[
    ['shelter_costs_owned','shelter_costs_rented']
]

In [9]:
data_types = sum_dwelling[
    ['single_detached_house', 'apartment_five_storeys_plus','movable_dwelling','semi_detached_house','row_house',
    'duplex', 'apartment_five_storeys_less', 'other_house']
]

In [10]:
### Data organsining for plots
df_mean = to_data[
    ['neighbourhood', 'average_house_value']
]

In [11]:
## Data for Dwelling Type Units per year
row = data_types.iloc[0]
row1 = data_types.iloc[1]
row2 = data_types.iloc[2]
row3 = data_types.iloc[3]

In [12]:
### Data for Sunburst Chart - Top 10 Expensive Neighbourhoods by Year
df_mean_reset = to_data.reset_index()
df_mean_reset_2001 = df_mean_reset.loc[df_mean_reset['year'] == 2001].sort_values('average_house_value', ascending=False)[0:10]
df_mean_reset_2006 = df_mean_reset.loc[df_mean_reset['year'] == 2006].sort_values('average_house_value', ascending=False)[0:10]
df_mean_reset_2011 = df_mean_reset.loc[df_mean_reset['year'] == 2011].sort_values('average_house_value', ascending=False)[0:10]
df_mean_reset_2016 = df_mean_reset.loc[df_mean_reset['year'] == 2016].sort_values('average_house_value', ascending=False)[0:10]

top10_richest_neighbourhoods_byyear = pd.concat([df_mean_reset_2001, df_mean_reset_2006, df_mean_reset_2011,df_mean_reset_2016])

In [13]:
## Bar Chart with Facet Row Data Reorder
avg_house_value_year = to_data[
    ['neighbourhood','average_house_value']
].reset_index()

## Panel Visualization Functions

In [110]:
""" Year 2001 Dwelling Types """

def dwelling_types_per_year(
    row = row.hvplot(kind='bar',title='Year 2001 Dwelling Types', color='m',rot=20, yformatter='%.0f')
):
    return row

In [95]:
""" Year 2006 Dwelling Types """

def dwelling_types_per_year_1(
    row1 = row1.hvplot(kind='bar',title='Year 2006 Dwelling Types',figsize=(12,6), color='c',rot=20,yformatter='%.0f')
):
    return row1

In [96]:
""" Year 2016 Dwelling Types """

def dwelling_types_per_year_2(
    row2 = row2.hvplot(kind='bar',title='Year 2011 Dwelling Types',figsize=(12,6), color='y',rot=20,yformatter='%.0f')
):
    return row2

In [97]:
""" Year 2016 Dwelling Types """

def dwelling_types_per_year_3(
    row3 = row3.hvplot(kind='bar',title='Year 2016 Dwelling Types',figsize=(12,6), color='b',rot=20,yformatter='%.0f')
):
    return row3

In [100]:
row_of_bar = pn.Row(dwelling_types_per_year,dwelling_types_per_year_1)
row_of_bar2 = pn.Row(dwelling_types_per_year_2,dwelling_types_per_year_3)

In [99]:
    """Neighbourhood Map"""
    
# Define Panel visualization functions
def neighbourhood_map(
    
    fig = px.scatter_mapbox(df_mean_neighbourhood_locations, lat="lat", lon="lon",   color="average_house_value",
                  color_continuous_scale=px.colors.cyclical.IceFire, size='average_house_value', zoom=10,mapbox_style='open-street-map', height=600, width=1200, title='Average House Value Scatterplot',
                            hover_data=['single_detached_house','apartment_five_storeys_plus','movable_dwelling','semi_detached_house', 'row_house', 'duplex', 'apartment_five_storeys_less',
                                                                      'other_house', 'shelter_costs_owned','shelter_costs_rented'])

):
        return fig

In [20]:
""" Top 10 Richest Neighbourhoods in Toronto"""

def top_10_richest_neighbourhoods(

top_10_richest = richest_neighbourhoods_sorted_top_10_housevalue.hvplot.bar(width=750,height=600, rot=60, 
                                                           title='Top 10 Most Expensive Neighbourhoods In Toronto',
                                                           ylabel='Price',
                                                          xlabel='Price', fontscale=1.2,yformatter="%.0f")
):
    return top_10_richest

In [21]:
"""Average house values by neighbourhood."""
    
    # YOUR CODE HERE!

def neighbourhood_avg_house_value(
    
    avg_house_value = df_mean.hvplot(x='year', y='average_house_value', width=1050, height=500, color='red',
                                                                  line_width= 5, groupby='neighbourhood', widget_location='top_left', yformatter='%.0f'
                                                                  ,xlabel='Year',ylabel='Average House Value Price', title='Average House Value')
    
):
        return avg_house_value

In [22]:
    """Average Shelter Owned and Rented Costs for all Toronto's neighbourhoods per year."""

    # YOUR CODE HERE!

# Create two line charts, one to plot the monthly shelter costs for owned dwelleing and 
## other for rented dwellings per year

def average_shelter_value(
    costs_mean_plot = costs_mean.hvplot(x='year', y=['shelter_costs_owned','shelter_costs_rented'], width=800, height=500, 
                            line_width= 5, groupby='neighbourhood',widget_location='top_left', yformatter='%.0f',xlabel='Year',ylabel='Average House Value Price',
                                       fontscale=1.5)
    
):
        return costs_mean_plot

In [23]:
 """ Number of dwelling types per year"""
    
def number_dwelling_types(
    
    number_dwelling_types_plot = to_data_reset_plot.hvplot.bar(x='year', y=['single_detached_house','apartment_five_storeys_plus','movable_dwelling',
                                               'semi_detached_house', 'row_house', 'duplex', 'apartment_five_storeys_less',
                                               'other_house'],
                                  stacked=False, width=900,height=600, rot=90,groupby='neighbourhood', widget_location='top_left',
                                  yformatter='%.0f"', xlabel= 'Year', ylabel = 'Dwelling Type Units', colormap='rainbow', fontscale=1.25)

):
    return number_dwelling_types_plot

In [24]:
""" Bar Chart Facet Row"""

def barchart_facet(
    fig = px.bar(avg_house_value_year, x='neighbourhood', y='average_house_value', color='average_house_value', facet_row='year', height=800, width=1000)
    
):
    return fig

In [25]:
""" Sunburst Chart for average House Value"""
    
def sunburst_house_chart(
    
    sunburst_chart = px.sunburst(top10_richest_neighbourhoods_byyear, path=['year','neighbourhood'],values='average_house_value', 
                             color='average_house_value', hover_data=['single_detached_house','apartment_five_storeys_plus','movable_dwelling','semi_detached_house', 'row_house', 'duplex', 'apartment_five_storeys_less',
                                                                      'other_house', 'shelter_costs_owned','shelter_costs_rented'],
                                 width=900,height=900)
):
    return sunburst_chart

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [108]:
# Create a Title for the Dashboard
# Define a welcome text
# Create a tab layout for the dashboard
# Create the main dashboard
# YOUR CODE HERE!

plot_as_column = pn.Column(
    ("# Average House Dashboard"),
    ("Welcome to an interaction Real Estate Analysis of Toronto Dashboard."),
    ("This dashborad includes an analysis of the average house value per neighouhood, sheltter costs for owned and rented properties per neighbourhood and the types of dwelling units."),
    ("To see the data justs click through the tabs on the top of the dashboard and it will direct you to the relevant information. Enjoy!")
    , neighbourhood_map
)

plot_as_column_row = pn.Column(row_of_bar, row_of_bar2)

plot_as_column_2 = pn.Column(
    ("# Shelter Costs Owned vs. Rented"),
    average_shelter_value
)

plot_as_column_3 = pn.Column(
    ("# Dwelling Type  Units"),
    ("This graph includes all dwelling type units per neighbourhood for the years 2001, 2006, 2011 and 2016."),
    number_dwelling_types, barchart_facet
)

plot_as_column_4 = pn.Column(
    ("# Average House Value"),
 barchart_facet, sunburst_house_chart
)

plot_as_column_5 = pn.Column(
    ("# Top 10 Richest Neighbourhoods"),
    ("Click year to enter that year data only. Hover over neighbourhoods to find the dwelling type units."),
    sunburst_house_chart, top_10_richest_neighbourhoods
)

plot_as_column_6 = pn.Column("# Top 10 Richest Neighbourhoods", "Click year to enter that year data only. Hover over neighbourhoods to find all combined data."
                      ,sunburst_house_chart,top_10_richest_neighbourhoods )


tabs = pn.Tabs(
    ## hello
    ("Welcome", plot_as_column),
    ("Dwelling Types Per Year", plot_as_column_row),
    ("Shelter Costs Owned vs. Rented", plot_as_column_2),
    ("Neighbourhood Analysis", plot_as_column_3),
    ("Top 10 Richest Neighbourhoods",plot_as_column_6)
)

## Serve the Panel Dashboard

In [107]:
tabs.servable()

Tabs
    [0] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Markdown(str)
        [3] Markdown(str)
        [4] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive14734')
    [1] Column
        [0] Row
            [0] Column
                [0] Column()
                [1] Row
                    [0] HoloViews(Bars, name='interactive10045')
            [1] Column
                [0] Column()
                [1] Row
                    [0] HoloViews(Bars, name='interactive10055')
        [1] Row
            [0] Column
                [0] Column()
                [1] Row
                    [0] HoloViews(Bars, name='interactive10066')
            [1] Column
                [0] Column()
                [1] Row
                    [0] HoloViews(Bars, name='interactive10076')
    [2] Column
        [0] Markdown(str)
        [1] Column
            [0] Column()
            [1] Row
                [0] Column
                    [0] Row
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighbourhood', options=['Agincourt North', ...], value='Annex', width=250)
                        [1] HSpacer()
                    [1] HoloViews(DynamicMap, widget_location='top_left')
    [3] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Column
            [0] Column()
            [1] Row
                [0] Column
                    [0] Row
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighbourhood', options=['Agincourt North', ...], value='Agincourt North', width=250)
                        [1] HSpacer()
                    [1] HoloViews(DynamicMap, widget_location='top_left')
        [3] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive14760')
    [4] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive14811')
        [3] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Bars, name='interactive14818')

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [28]:
# neighbourhood_map().show()

In [29]:
# create_bar_chart(data, title, xlabel, ylabel, color)

# # Bar chart for 2001
# create_bar_chart(df_dwelling_units.loc[2001], "Dwelling Types in Toronto in 2001", "2001", "Dwelling Type Units", "red")

# # Bar chart for 2006
# create_bar_chart(df_dwelling_units.loc[2006], "Dwelling Types in Toronto in 2006", "2006", "Dwelling Type Units", "blue")

# # Bar chart for 2011
# create_bar_chart(df_dwelling_units.loc[2011], "Dwelling Types in Toronto in 2011", "2011", "Dwelling Type Units", "orange")

# # Bar chart for 2016
# create_bar_chart(df_dwelling_units.loc[2016], "Dwelling Types in Toronto in 2016", "2016", "Dwelling Type Units", "magenta")

In [30]:
# create_line_chart(data, title, xlabel, ylabel, color)

# # Line chart for owned dwellings
# create_line_chart(df_avg_costs["shelter_costs_owned"], "Average Monthly Shelter Cost for Owned Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "blue")

# # Line chart for rented dwellings
# create_line_chart(df_avg_costs["shelter_costs_rented"], "Average Monthly Shelter Cost for Rented Dwellings in Toronto", "Year", "Avg Monthly Shelter Costs", "orange")

In [31]:
# average_house_value()

In [32]:
# average_value_by_neighbourhood()

In [33]:
# number_dwelling_types()

In [34]:
# average_house_value_snapshot()

In [35]:
# top_most_expensive_neighbourhoods()

In [36]:
# sunburts_cost_analysis()

In [37]:
sum_dwelling = to_data.groupby(by=['year']).sum()
sum_dwelling.head()

,single_detached_house,apartment_five_storeys_plus,movable_dwelling,semi_detached_house,row_house,duplex,apartment_five_storeys_less,other_house,average_house_value,shelter_costs_owned,shelter_costs_rented
year,,,,,,,,,,,
2001,300930,355015,75,90995,52355,23785,116900,3040,40583604,118563,152031
2006,266860,379400,165,69430,54690,44095,162850,1335,59368353,184352,129558
2011,274940,429220,100,72480,60355,44750,163895,2165,74259461,202750,142771
2016,269680,493270,95,71200,61565,48585,165575,2845,92969566,246584,175885
